In [ ]:
from torchvision.datasets import VOCDetection
from torch.utils.data import Dataset
import torch
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import ToTensor
from tqdm import tqdm
import numpy as np
import cv2 as cv

from dataset import YoloVOCDataset

In [ ]:
IMG_SIZE=416

voc_ds=VOCDetection(root='data',year='2012',image_set='train',download=False)
classdict=set()
anchor_boxex=[]
for _,label in tqdm(voc_ds, desc="数据集处理中"):
    for obj in label['annotation']['object']:
        classdict.add(obj['name'])
        xmin,ymin=int(obj['bndbox']['xmin']),int(obj['bndbox']['ymin'])
        xmax,ymax=int(obj['bndbox']['xmax']),int(obj['bndbox']['ymax'])
        w=xmax-xmin
        h=ymax-ymin
        anchor_boxex.append([w,h])
names=sorted(list(classdict))
id2name={i:c for i,c in enumerate(names)}
name2id={c:i for i,c in id2name.items()}